# Activity Fronts in Spatially-Extended Stochastic Lotka-Volterra Model

In this example, we examine activity fronts in the Lotka-Volterra system on a lattice. Activity fronts are waves of predator particles invading into a region filled with prey that was previously predator-free. The likely reason for the previous absence of predators is that an activity front has "ravaged" this region before and only now prey have come back here in sufficient numbers to support another wave of predators.

To start, we need to install and load the `pymes` library. We only load what we need so not to clutter our environment.

In [ ]:
from pymes import World, Species, BirthReaction, DeathReaction, PredationBirthReaction, Hop

Next, we set up the world. This is very similar to the basic example. It again involves setting up the species (the predator A and the prey B) and the reactions (B can give birth, A can die and predate upon B). The main differences are that we make the world bigger to study activity fronts, and that we fill the lattice with prey particles that can neither reproduce nor move. This is an intentional oversimplification. We leave it as an exercise to make it more realistic. The following are the parameters of the simulation:
* `size`: This is the size of the 2D lattice and is specified as a tuple of two integers (the width and the height) like so `(500, 500)`. This value is a good size to study activity fronts, but it also means that single runs can take a while to complete.
* `sigma`: This is the prey (species "B") birth rate. It regulates how quickly prey will reproduce. We set this to zero here, since we don't want prey to reproduce for now (but there is no reason to not turn on reproduction later).
* `mu`: This is the predator (species "A") death rate. It regulates how quickly predators are removed from the system.
* `lam`: This is the predation rate. It regulates how efficiently predators (species "A") consume prey (species "B") and reproduce at the same time. We will vary this below to see how it affects the speed of activity fronts.
* `hA` and `hB`: These are the hop rates of the species. We let the predators move freely, but will keep the prey stationary. Again, we leave it as an exercise to turn on prey movement and see how that affects activity fronts.
* `nA` and `nB`: We set up the system such that every lattice site is filled with `nB=5` prey particles. We then seed the center of the lattice with `nA=10` predators.
* `T`: Specifies the end time of the simulation, i.e. how many Monte Carlo steps will be performed.

In [ ]:
# The simulation parameters
size = (200, 200)
sigma: float = 0.0
mu: float = 0.2
lam: float = 0.5
hA: float = 1.0
hB: float = 0.0
nA: int = 10
nB: int = 5
T: int = 50

# Predator and prey species
A = Species("A")
B = Species("B")

# The reactions
reactions = {
    A: [
        PredationBirthReaction(A, B, lam),
        DeathReaction(A, mu),
    ],
    B: [
        BirthReaction(B, sigma),
    ]
}

# Initialization of the world
world = World(
    size=size,
    initial_densities={B: 0.0, A: 0.0},
    hops={A: Hop(A, hA), B: Hop(B, hB)},
    reactions=reactions,
)

# Let's fill the grid completely with B particles
for site in world.lattice.sites:
    for _ in range(5):
        B.create_occupant(site)

# And seed the center lattice site with A particles
middle = world.lattice.sites[size[1]//2 * size[0] + size[0]//2]
for _ in range(10):
    A.create_occupant(middle)

Again, we're ready to run the simulations. This will perform `T=200` Monte Carlo steps and record the state of the lattice at each step to see the activity front develop.

In [ ]:
from tqdm.notebook import tqdm
tqdm.monitor_interval = 0

# Initialize list that will hold a view of the lattice state at each step.
# The first entry shows the initial distribution of species site occupants.
numbers = [world.asarrays()]

# Iterate over `T` time steps
# `tqdm` is a tool to display a progress bar.
for _ in tqdm(range(T), smoothing=0, desc="Simulation progress"):
    # Run a single time step
    world.step()
    # Save the state of the lattice (distribution of occupants)
    numbers.append(world.asarrays())

Let's visualize the results. As in the basic example. the code in the cell below displays an interactive graph where you can see the state of the lattice at each step (left side), together with a plot of the species densities over time (right side). 

In [ ]:
# Display matplotlib figures inline (not the interactive notebook extension)
%matplotlib inline
# Load modules
from matplotlib.figure import Figure
from ipywidgets import interact
import numpy as np
from IPython.display import display

# Initialize figure and axes
simul_fig = Figure(figsize=(8, 4), dpi=150)
simul_ax = simul_fig.add_axes([0, 0, 0.5, 1])
# We don't want any decorations on the axes.
simul_ax.axis('off')
simul_plot = simul_fig.add_axes([0.6, 0.1, 0.4, 0.8])

# Extract densities from species distribution arrays
N = size[0]*size[1]
A_densities, B_densities = zip(*[(entry["A"].sum()/N, entry["B"].sum()/N) for entry in numbers])

# Plot the densities
simul_plot.plot(A_densities, color="red", label="A")
simul_plot.plot(B_densities, color="blue", label="B")
# Label the plot
simul_plot.set_yscale("log", base=10)
simul_plot.set_ylabel("Species density [occupants/site]")
simul_plot.set_xlabel("Time [MC steps]")

# This code displays the correct view of the lattice given the MC step
mappable = None
vline = None
def show_image(MC_step=0):
    global mappable, vline  # Hackish, but easy, avoid global if possible.

    # Load array of lattice occupant numbers
    arrays = numbers[MC_step]
    # Create a red and blue image
    # `image` is a WxHx3 image where the three channels at the end correspond to red-green-blue
    image = np.zeros((size[0], size[1], 3), dtype=np.uint8)
    # Make a pixel that has at least one predator red, at least one prey blue and purple if both are present.
    # Leave the green channel alone.
    image[:, :, 0] = 255*(arrays["A"] > 0)
    image[:, :, 2] = 255*(arrays["B"] > 0)
    if mappable is None:
        # We need to create the mappable first, afterwards we can just load data into it.
        mappable = simul_ax.imshow(image)
        # Similar for the vline indicating the MC step we're at.
        vline = simul_plot.axvline(0, color="black")
    else:
        # Load data into mappable and vline.
        mappable.set_data(image)
        vline.set_xdata([MC_step])

    # Finally, display the updated figure.
    display(simul_fig)

# THis lets us have a slider to select the MC step to display.
interact(show_image, MC_step=(0, len(numbers)-1))
None

For the first time step, you should see a sea of blue prey on the lattice, with a tiny red dot in the center (representing the predators). As you move the slider to the right, you see an activity front develop and spread outward. The graph on the right again shows the total density of predators and prey over time. With the default settings, the prey do not reproduce, hence as they are consumed, they density of prey diminishes. As the predators move outwards, they consume prey and reproduce forming a circular front. The density of predators increases therefore in a quadratic manner as the front expands. This continues until the front merges with itself via the periodic boundary of the lattice and the front distorts and eventually vanishes.

We now want to track the front as it moves outwards. Symmetry dictates that the front is circular on average. Therefore we can simply radially average the number of occupants of each species to calculate the density as a function of the distance from the lattice center. The function defined below accomplishes this.

In [ ]:
def radial_avg(a):
    """Given a two dimensional array `a`, calculate the radial average measured from the center."""
    # First we determine the distance `r` of each lattice site from the center of the lattice.
    xx, yy = np.indices(a.shape)
    r = np.sqrt((xx-a.shape[0]//2)**2 + (yy-a.shape[1]//2)**2).astype(int)

    # Next, bin according to the integer distance values and weigh them according to the lattice occupancy.
    tbin = np.bincount(r.ravel(), a.ravel())
    # We also need to know the unweighed number of lattice sites in each bin.
    nr = np.bincount(r.ravel())

    # Calculate and return the average density in each distance bin.
    return tbin / nr

In [ ]:
# The time point `t` we choose to visualize the front at.
t = 30

# Radially average the predator and prey species densities
# This is done so that we can identify how far from the lattice center the activity wave has travelled
a = radial_avg(numbers[t]['A'])
b = radial_avg(numbers[t]['B'])

# Initialize the figure and plot the radially averaged densities
fig = Figure()
plot = fig.subplots(1, 1)
plot.plot(a, color='red')
plot.plot(b, color='blue')

plot.set_xlabel('Distance from lattice center [sites]')
plot.set_ylabel('Radially averaged species density [occupants/site]')

plot.text(0.02, 0.98, f't={t}', ha='left', va='top', transform=plot.transAxes)

display(fig)

We see that the predators form a clear front with a density maximum at a radius of around `r=80` at time point `t=50`. They prey front precedes this, forming a neat S-shaped curve with an inflection point slightly further outward compared to the predator maximum. The distance between the predator maximum and the prey inflection point surely depends on both the predation rate `lam` and the predator death rate `mu`. We leave an investigation of this as an advanced exercise to the reader.

In order to find the position of the front as a function of time, we opt to fit the logistic function to the radial density of the prey (likely any other sigmoid function would work as well). The function below does exactly that. The fit extracts the location of the inflection point `x0`.

In [ ]:
from scipy.optimize import curve_fit
from scipy.special import expit as logistic_function

def extract_front_position(radial_density):
    # Here, we use the radially averaged density of B particles and fit a logitistic function to it.
    # This way, we can extract the location of the inflection point of this function at each time point.
    x0 = curve_fit(lambda x, x0: 5*logistic_function(x-x0), np.arange(0, radial_density.shape[0]), radial_density)[0]
    return x0

In order for the fit to work well, we need to exclude the time points in which no prey are left (otherwise the least squares fitting algorithm will return an error). We also filter out front positions which indicate that the front is not circular anymore because it has merged through the periodic lattice boundary. This is accomplished by rejecting extracted front positions that are less than half the lattice size from the center.

In [ ]:
# Extract the front position for all time points for which there are still B particles on the lattice.
# We also need to exclude the first time point, since no front has yet developed.
pos = np.array([extract_front_position(radial_avg(timepoint['B'])) for timepoint in numbers[1:] if timepoint['B'].max() > 0.0])

# Also exclude positions that are greater than the front radius that can be accommodated on the lattice
pos = pos[pos < size[0] // 2]

# Initialize the figure and plot the radially averaged densities
fig = Figure()
pos_plot, speed_plot = fig.subplots(2, 1, sharex=True)
pos_plot.plot(pos, color='black')
speed_plot.plot(np.diff(pos), color='black')

speed_plot.set_xlabel('Time [MC step]')
pos_plot.set_ylabel('Front position [sites]')
speed_plot.set_ylabel('Front speed [sites / MC step]')
speed_plot.axhline(np.diff(pos).mean(), color='black', ls='--')

display(fig)

We see that the front position linearly increases as a function of time. The speed (calculated as the position change between each time step) is relatively constant. The average speed is around 1.9 lattice sites per time step.

Next, we'd like to see how this average front speed changes as a function of the predation rate `lam`. We simply wrap the simulation setup in a function which we can run with different values of the parameters, see below:

In [ ]:
def run_simulation(
    size: tuple[float, float] = (150, 150),
    mu: float = 0.2,
    lam: float = 0.5,
    hA: float = 1.0,
    nB: int = 5,
    nA: int = 10,
    T: int = 50
):
    # Predator and prey species
    A = Species("A")
    B = Species("B")

    # The reactions
    reactions = {
        A: [
            PredationBirthReaction(A, B, lam),
            DeathReaction(A, mu),
        ],
        B: []   # We leave B as non-reproductive here.
    }

    # Initialization of the world
    world = World(
        size=size,
        initial_densities={B: 0.0, A: 0.0},  # We place particles of each species manually below.
        hops={A: Hop(A, hA)},  # Only A is allowed to move, B is stationary.
        reactions=reactions,
    )

    # Let's fill the grid completely with B particles
    for site in world.lattice.sites:
        for _ in range(nB):
            B.create_occupant(site)

    # And seed the center lattice site with A particles
    middle = world.lattice.sites[size[1]//2 * size[0] + size[0]//2]
    for _ in range(nA):
        A.create_occupant(middle)

    # Initialize list that will hold a view of the lattice state at each step.
    # The first entry shows the initial distribution of species site occupants.
    numbers = [world.asarrays()]

    # Iterate over `T` time steps
    # `tqdm` is a tool to display a progress bar.
    for _ in tqdm(range(T), smoothing=0, desc="Simulation progress"):
        # Run a single time step
        world.step()
        # Save the state of the lattice (distribution of occupants)
        numbers.append(world.asarrays())
    
    return numbers

This allows us to then execute the function `run_simulation` with different values of `lam`. We are simply repeating the steps we had done above again for each value of `lam`. Running this might well take several hours, depending on the hardware.

In [ ]:
# List of predation rate values we want to extract the speed for.
lam_list = np.linspace(0.25, 1.0, 10)

# These lists will hold the average speed and its standard deviation for each value of `lam` respectively.
average_speed = []
std_speed = []

# Now loop over all values of `lam`.
for lam in tqdm(lam_list):
    # Run the simulation.
    numbers = run_simulation(lam=lam)

    # Extract the front position for all time points for which there are still B particles on the lattice.
    # We also need to exclude the first time point, since no front has yet developed.
    pos = np.array([extract_front_position(radial_avg(timepoint['B'])) for timepoint in numbers[1:] if timepoint['B'].max() > 0.0])

    # Also exclude positions that are greater than the front radius that can be accommodated on the lattice
    pos = pos[pos < size[0] // 2]

    speed = np.diff(pos)
    average_speed.append(speed.mean())
    std_speed.append(speed.std())

In [ ]:
# Bring the data into a plottable format
lams = np.array(lam_list[:len(average_speed)])
average_speeds = np.array(average_speed)
std_speeds = np.array(std_speed)

# Plot the average speed, with the standard deviation as shaded area around it 
fig = Figure()
ax = fig.subplots(1, 1)
ax.plot(lams, average_speeds)
ax.fill_between(lams, average_speeds - std_speeds, average_speeds + std_speeds, color='C0', alpha=0.25)

ax.set_xlabel(r'Predation rate $\lambda$')
ax.set_ylabel('Activity front speed [sites/MC step]')
display(fig)

We can clearly see that the front speed increases with the predation rate. While this relationship seems linear, there is a slight curvature. Please see [Dobramysl, Tauber, Spatial Variability Enhances Species Fitness in Stochastic Predator-Prey Interactions] for a deeper analysis of this topic. We leave the exploration of the front speed dependence on the other parameters as an exercise to the reader.